In [1]:
import pandas as pd
import json
import gzip
from collections import defaultdict, Counter
import torch
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import tqdm 

# ==========================================
# CONFIGURATION
# ==========================================
PATH_REVIEWS = '/kaggle/input/books-dataset/Books_5.json'        
PATH_META = '/kaggle/input/metadata/meta_Books.json'        
MIN_INTERACTIONS = 10                
RATING_THRESHOLD = 3.0       

MAX_LINES_TO_READ = float('inf') 

def parse_all(path):
    """Lecture intégrale des lignes du fichier, sans limite."""
    # Cette fonction est utilisée pour la lecture intégrale des fichiers
    if path.endswith('.gz'):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)
    else:
        with open(path, 'r') as f:
            for l in f:
                yield json.loads(l)

In [2]:
# ==========================================
# 1. TRAITEMENT DES INTERACTIONS 
# ==========================================
print("--- 1. Phase K-Core : Identification des IDs valides ---")

# --- Étape 0 : Statistiques brutes du fichier ---
raw_lines_count = 0
total_interactions = 0 # Interactions positives (> seuil)
user_counts = Counter()
item_counts = Counter()

print("Lecture du fichier pour les comptes initiaux...")
for review in parse_all(PATH_REVIEWS):
    raw_lines_count += 1
    
    # Filtrage par note (Rating Threshold)
    if review.get('overall', 0.0) > RATING_THRESHOLD:
        reviewerID = review.get('reviewerID')
        asin = review.get('asin')
        if reviewerID and asin:
            user_counts[reviewerID] += 1
            item_counts[asin] += 1
            total_interactions += 1

# Affichage des statistiques avant le filtre K-Core
print(f"\n--- Statistiques avant filtrage ---")
print(f"Nombre total de lignes dans le fichier (Brut) : {raw_lines_count:,}")
print(f"Nombre d'interactions positives (> {RATING_THRESHOLD}) : {total_interactions:,}")
print(f"Nombre d'utilisateurs uniques (Positifs) : {len(user_counts):,}")
print(f"Nombre d'items uniques (Positifs) : {len(item_counts):,}")
print("-" * 40)

--- 1. Phase K-Core : Identification des IDs valides ---
Lecture du fichier pour les comptes initiaux...

--- Statistiques avant filtrage ---
Nombre total de lignes dans le fichier (Brut) : 27,164,983
Nombre d'interactions positives (> 3.0) : 22,949,531
Nombre d'utilisateurs uniques (Positifs) : 1,849,283
Nombre d'items uniques (Positifs) : 703,533
----------------------------------------


In [3]:
# --- Étape 2 : Filtrage itératif K-Core (10-core) ---
print(f"Application du filtre {MIN_INTERACTIONS}-core itératif...")
valid_users = set(user_counts.keys())
valid_items = set(item_counts.keys())

while True:
    old_user_count = len(valid_users)
    old_item_count = len(valid_items)

    # 1. Filtrer les utilisateurs (basé sur l'étape 1)
    new_valid_users = {u for u in valid_users if user_counts.get(u, 0) > MIN_INTERACTIONS}
    
    # 2. Filtrer les items (basé sur l'étape 1)
    new_valid_items = {i for i in valid_items if item_counts.get(i, 0) > MIN_INTERACTIONS}
    
    # 3. Mettre à jour les ensembles
    valid_users = new_valid_users
    valid_items = new_valid_items
    
    # 4. Vérifier la condition d'arrêt
    if len(valid_users) == old_user_count and len(valid_items) == old_item_count:
        break
        
    # Recalculer les comptes d'interactions restantes après filtrage
    user_counts.clear()
    item_counts.clear()
    
    # RELIRE le fichier pour recalculer les comptes des entités restantes
    for review in parse_all(PATH_REVIEWS):
        if review.get('overall', 0.0) > RATING_THRESHOLD:
            reviewerID = review.get('reviewerID')
            asin = review.get('asin')
            
            if reviewerID in valid_users and asin in valid_items:
                user_counts[reviewerID] += 1
                item_counts[asin] += 1

print(f"\n K-Core Terminé.")
print(f"Utilisateurs uniques restants : {len(valid_users):,}")
print(f"Items (Livres) uniques restants : {len(valid_items):,}")

Application du filtre 10-core itératif...

 K-Core Terminé.
Utilisateurs uniques restants : 392,089
Items (Livres) uniques restants : 215,039


In [4]:

# --- Étape 3 : Construction du DataFrame UI final ---
ui_data = []
final_interactions = 0

print("\nConstruction du DataFrame UI final...")
for review in parse_all(PATH_REVIEWS):
    reviewerID = review.get('reviewerID')
    asin = review.get('asin')
    
    # Ne garder que les interactions positives des IDs valides
    if review.get('overall', 0.0) > RATING_THRESHOLD and \
       reviewerID in valid_users and asin in valid_items:
        ui_data.append([reviewerID, asin])
        final_interactions += 1
        
# Création du DataFrame UI
df_ui = pd.DataFrame(ui_data, columns=['reviewerID', 'asin']).drop_duplicates()
print(f"Interactions finales (Clean) : {len(df_ui):,}")
valid_item_ids = set(df_ui['asin'].unique())


Construction du DataFrame UI final...
Interactions finales (Clean) : 11,544,524


In [5]:
# ==========================================
# 1.5. SÉPARATION TRAIN / VALIDATION / TEST (80% / 10% / 10%)
# ==========================================
print("\n--- 1.5. Séparation du Dataset UI (Train/Validation/Test) ---")

# Mélanger le DataFrame pour assurer la randomisation
df_ui = df_ui.sample(frac=1, random_state=42).reset_index(drop=True)

# Définir les pourcentages de division
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1 

df_train_list = []
df_val_list = []
df_test_list = []

# Séparation par utilisateur (User-based split)
for _, group in df_ui.groupby('reviewerID'):
    n = len(group)
    
    # Calcul des tailles
    n_test = int(TEST_RATIO * n)
    n_val = int(VAL_RATIO * n)
    # Le reste est pour l'entraînement
    n_train = n - n_test - n_val
    
    # Séquence des indices
    test_indices = group.index[:n_test]
    val_indices = group.index[n_test:n_test + n_val]
    train_indices = group.index[n_test + n_val:]
    
    # Ajout aux listes
    df_test_list.append(group.loc[test_indices])
    df_val_list.append(group.loc[val_indices])
    df_train_list.append(group.loc[train_indices])

# Concaténation des résultats
df_train = pd.concat(df_train_list)
df_val = pd.concat(df_val_list)
df_test = pd.concat(df_test_list)

# Vérifications
total_rows = len(df_ui)
print(f"Total Interactions: {total_rows:,}")
print(f"Train Set : {len(df_train):,} ({len(df_train)/total_rows:.2%})")
print(f"Validation Set : {len(df_val):,} ({len(df_val)/total_rows:.2%})")
print(f"Test Set : {len(df_test):,} ({len(df_test)/total_rows:.2%})")


--- 1.5. Séparation du Dataset UI (Train/Validation/Test) ---
Total Interactions: 11,544,524
Train Set : 9,541,364 (82.65%)
Validation Set : 1,001,580 (8.68%)
Test Set : 1,001,580 (8.68%)


In [6]:
# ==========================================
# 2. TRAITEMENT DU KNOWLEDGE GRAPH (KG) 
# ==========================================
print("--- 2. Construction du Knowledge Graph (KG) à partir de Metadata ---")

kg_triplets = [] 
triplet_stats = defaultdict(int)
items_found = 0

for meta in parse_all(PATH_META):
    asin = meta.get('asin')
    
    # 1.  Filtrage par la liste valid_item_ids (items ayant passé le 10-core)
    if asin in valid_item_ids:
        items_found += 1
        
        # RELATION 1: Item -> Brand (Marque/Éditeur)
        if 'brand' in meta and meta['brand'] and meta['brand'].strip() != '':
            brand = meta['brand'].strip()
            if brand.startswith("Visit Amazon's"):
                brand = brand.replace("Visit Amazon's", "").replace(" Page", "").strip()
            if brand:
                kg_triplets.append((asin, 'brand', brand))
                triplet_stats['brand'] += 1
                
        # RELATION 2: Item -> Categories
        if 'category' in meta:
            for cat in meta['category']: 
                cat = cat.strip()
                if cat and cat.lower() not in ['books', 'all departments', '']:
                    kg_triplets.append((asin, 'category', cat))
                    triplet_stats['category'] += 1
        
        # RELATION 3: Item -> Item (also_bought)
        if 'also_buy' in meta and isinstance(meta['also_buy'], list):
            for related_asin in meta['also_buy']:
                # On ajoute la relation UNIQUEMENT si l'item lié est aussi dans notre dataset final
                if related_asin in valid_item_ids:
                    kg_triplets.append((asin, 'also_bought', related_asin))
                    triplet_stats['also_bought'] += 1

        # RELATION 4: Item -> Item (also_viewed)
        if 'also_view' in meta and isinstance(meta['also_view'], list):
            for related_asin in meta['also_view']:
                if related_asin in valid_item_ids:
                    kg_triplets.append((asin, 'also_viewed', related_asin))
                    triplet_stats['also_viewed'] += 1

# Conversion en DataFrame final
df_kg = pd.DataFrame(kg_triplets, columns=['head', 'relation', 'tail'])

relation_encoder = LabelEncoder()
relation_encoder.fit(df_kg['relation'])

# --- Affichages des résultats ---
print(f"\n--- Statistiques du Knowledge Graph Final ---")
print(f"Items du Graphe UI (valid_item_ids) trouvés dans Meta : {items_found:,}")
print(f"Total Triples KG extraits : {len(df_kg):,}")
print(f"Relations uniques dans le KG : {df_kg['relation'].nunique()}")

print("\nRépartition des Triplets :")
for rel, count in triplet_stats.items():
    print(f"- {rel.ljust(15)} : {count:,} triplets")
    
print("\n--- Aperçu des 5 premiers Triplets KG : ---")
print(df_kg.head())


--- 2. Construction du Knowledge Graph (KG) à partir de Metadata ---

--- Statistiques du Knowledge Graph Final ---
Items du Graphe UI (valid_item_ids) trouvés dans Meta : 215,073
Total Triples KG extraits : 5,463,300
Relations uniques dans le KG : 4

Répartition des Triplets :
- brand           : 214,390 triplets
- category        : 418,646 triplets
- also_viewed     : 1,381,394 triplets
- also_bought     : 3,448,870 triplets

--- Aperçu des 5 premiers Triplets KG : ---
         head     relation                  tail
0  0001050230        brand   William Shakespeare
1  0001050230     category  Literature & Fiction
2  0001050230     category        Dramas & Plays
3  0001050230  also_viewed            0140714596
4  0001050230  also_viewed            0140714642


In [7]:

# ==========================================
# 3. INDEXATION ET CRÉATION DES TENSEURS
# ==========================================
print("--- 3. Indexation et Création des Tenseurs ---")

# 3.1 INITIALISATION ET FIT DES ENCODEURS

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
entity_encoder = LabelEncoder()

# A. Items (ASINs) : ENCODEUR GLOBAL FIT SUR TOUS LES ITEMS VALIDES
all_items = sorted(list(valid_item_ids))
item_encoder.fit(all_items)

# B. Utilisateurs (reviewerID) : ENCODEUR GLOBAL FIT SUR TOUS LES UTILISATEURS VALIDES

users_train = set(df_train['reviewerID'].unique())
users_val = set(df_val['reviewerID'].unique())
users_test = set(df_test['reviewerID'].unique())

# Utiliser l'opérateur d'union de set pour combiner tous les utilisateurs uniques
all_users = sorted(list(users_train.union(users_val).union(users_test)))

user_encoder.fit(all_users)

# C. Entités KG (Tails qui NE SONT PAS des ASINs)
kg_entity_tails = df_kg[~df_kg['relation'].str.contains('also_')]['tail']
entity_encoder.fit(kg_entity_tails.astype(str))

# 3.2 APPLICATION ET CRÉATION DES OFFSETS

NUM_USERS = len(user_encoder.classes_)
NUM_ITEMS = len(item_encoder.classes_)
NUM_RELATIONS = len(relation_encoder.classes_)
NUM_ENTITIES_KG = len(entity_encoder.classes_)

# Décalages pour créer un ID global unique
ITEM_OFFSET = NUM_USERS
ENTITY_OFFSET = NUM_USERS + NUM_ITEMS
TOTAL_NODES = NUM_USERS + NUM_ITEMS + NUM_ENTITIES_KG

print(f"\n- Nodes Totaux : {TOTAL_NODES:,}")
print(f"  - Utilisateurs : 0 à {NUM_USERS - 1:,}")
print(f"  - Items (ASINs) : {ITEM_OFFSET:,} à {ITEM_OFFSET + NUM_ITEMS - 1:,}")
print(f"  - Entités KG : {ENTITY_OFFSET:,} à {TOTAL_NODES - 1:,}")


--- 3. Indexation et Création des Tenseurs ---

- Nodes Totaux : 691,559
  - Utilisateurs : 0 à 392,088
  - Items (ASINs) : 392,089 à 607,127
  - Entités KG : 607,128 à 691,558


In [ ]:
# 3.3 CONVERSION EN TENSEURS PYTORCH

def create_ui_tensor(df, item_offset, bidirectional=False):
    """Crée les tenseurs UI (User-Item) avec les indices et l'offset."""
    u_idx = user_encoder.transform(df['reviewerID'])
    i_idx = item_encoder.transform(df['asin']) + item_offset
    
    u_tensor = torch.tensor(u_idx, dtype=torch.long)
    i_tensor = torch.tensor(i_idx, dtype=torch.long)
    
    edge_index = torch.stack([u_tensor, i_tensor], dim=0)
    
    if bidirectional:
        # Ajoute les arêtes Item -> User (nécessaire pour le training sur GNN)
        edge_index_iu = torch.stack([i_tensor, u_tensor], dim=0)
        edge_index = torch.cat([edge_index, edge_index_iu], dim=1)
        
    return edge_index

# Tenseur UI TRAIN (Bidirectionnel pour le GNN)
edge_index_train = create_ui_tensor(df_train, ITEM_OFFSET, bidirectional=True)
print(f"Tenseur UI TRAIN créé : {edge_index_train.shape} (Bidirectionnel)")

# Tenseurs UI VALIDATION et TEST (Unidirectionnels pour l'évaluation)
edge_index_val = create_ui_tensor(df_val, ITEM_OFFSET, bidirectional=False)
edge_index_test = create_ui_tensor(df_test, ITEM_OFFSET, bidirectional=False)
print(f"Tenseur UI VAL créé : {edge_index_val.shape}")
print(f"Tenseur UI TEST créé : {edge_index_test.shape}")


Tenseur UI TRAIN créé : torch.Size([2, 19082728]) (Bidirectionnel)
Tenseur UI VAL créé : torch.Size([2, 1001580])
Tenseur UI TEST créé : torch.Size([2, 1001580])


In [ ]:
# 3.3  TENSEUR KG OPTIMISÉ 

print("Mapping des indices KG .")

# 1. Pré-calculer les dictionnaires de mapping pour une recherche instantanée (O(1))
item_map = {asin: idx + ITEM_OFFSET for idx, asin in enumerate(item_encoder.classes_)}
entity_map = {ent: idx + ENTITY_OFFSET for idx, ent in enumerate(entity_encoder.classes_)}
rel_map = {rel: idx for idx, rel in enumerate(relation_encoder.classes_)}

# 2. Mapper les Têtes (Head) et les Relations de façon vectorisée
df_kg['head_idx'] = df_kg['head'].map(item_map)
df_kg['rel_idx'] = df_kg['relation'].map(rel_map)

# 3. Mapper les Queues (Tail) de façon intelligente
# On sépare les relations "also_" des autres pour utiliser le bon dictionnaire
is_also_rel = df_kg['relation'].str.contains('also_')

# Créer une colonne vide pour les indices de queue
df_kg['tail_idx'] = np.nan

# Appliquer le mapping d'items pour les relations also_bought/viewed
df_kg.loc[is_also_rel, 'tail_idx'] = df_kg.loc[is_also_rel, 'tail'].map(item_map)

# Appliquer le mapping d'entités pour les autres (brand, category)
df_kg.loc[~is_also_rel, 'tail_idx'] = df_kg.loc[~is_also_rel, 'tail'].astype(str).map(entity_map)

# Vérification : supprimer les éventuels NaN si un tail n'a pas pu être mappé
df_kg = df_kg.dropna(subset=['tail_idx'])
df_kg['tail_idx'] = df_kg['tail_idx'].astype(int)

# 4. Conversion finale en tenseurs
h_tensor = torch.tensor(df_kg['head_idx'].values, dtype=torch.long)
t_tensor = torch.tensor(df_kg['tail_idx'].values, dtype=torch.long)
r_tensor = torch.tensor(df_kg['rel_idx'].values, dtype=torch.long)

edge_index_kg = torch.stack([h_tensor, t_tensor], dim=0)
edge_type_kg = r_tensor

print(f" Tenseur KG créé: {edge_index_kg.shape }")

Mapping des indices KG .
 Tenseur KG créé: torch.Size([2, 5463300])


In [ ]:
# ==========================================
# 3.4 SAUVEGARDE DES TENSEURS ET MÉTADONNÉES
# ==========================================

# 1. Rassembler toutes les données importantes
data_tensors = {
    # Dimensions
    'num_nodes': TOTAL_NODES,
    'num_users': NUM_USERS,
    'num_items': NUM_ITEMS,
    'num_relations': NUM_RELATIONS,
    
    # Tenseurs UI
    'edge_index_train': edge_index_train, # Train (Bidirectionnel)
    'edge_index_val': edge_index_val,     # Validation (Unidirectionnel)
    'edge_index_test': edge_index_test,   # Test (Unidirectionnel)
    
    # Tenseurs KG
    'edge_index_kg': edge_index_kg, 
    'edge_type_kg': edge_type_kg,   
    
    # Encodeurs (pour le décodage)
    'user_encoder': user_encoder,
    'item_encoder': item_encoder,
    'entity_encoder': entity_encoder,
    'relation_dict': dict(zip(relation_encoder.classes_, range(NUM_RELATIONS)))
}

# 2. Définir le nom du fichier de sortie
OUTPUT_FILE = '/kaggle/working/amazon_kgcfrec_total_data.pt' 

# 3. Sauvegarde PyTorch
print(f"\nSauvegarde de l'objet data_tensors (taille totale : {len(data_tensors)} éléments) dans '{OUTPUT_FILE}'...")

try:
    torch.save(data_tensors, OUTPUT_FILE)
    print(f" Sauvegarde terminée. Le fichier '{OUTPUT_FILE}' est prêt pour l'entraînement.")

except Exception as e:
    print(f" Erreur lors de la sauvegarde PyTorch : {e}")


Sauvegarde de l'objet data_tensors (taille totale : 13 éléments) dans '/kaggle/working/amazon_kgcfrec_total_data.pt'...
 Sauvegarde terminée. Le fichier '/kaggle/working/amazon_kgcfrec_total_data.pt' est prêt pour l'entraînement.


In [ ]:

# 1. Chemin du fichier 
FILE_PATH = '/kaggle/working/amazon_kgcfrec_total_data.pt'

print(f"--- Chargement du fichier : {FILE_PATH} ---\n")

try:
    # 2. Chargement avec PyTorch
    loaded_data = torch.load(FILE_PATH, map_location=torch.device('cpu'), weights_only=False)
    
    print(" Fichier chargé avec succès !\n")
    print(f"{'CLÉ (Nom du variable)':<25} | {'TYPE':<15} | {'DÉTAIL / FORME'}")
    print("-" * 70)

    # 3. Inspection de chaque élément
    for key, value in loaded_data.items():
        val_type = type(value).__name__
        
        detail = ""
        if isinstance(value, int):
            detail = f"{value:,}" # Affiche le nombre formaté
        elif torch.is_tensor(value):
            detail = f"Shape: {list(value.shape)}"
        elif "Encoder" in str(type(value)):
            # Pour les encodeurs sklearn, on affiche le nombre de classes
            detail = f"Classes: {len(value.classes_):,}"
        elif isinstance(value, dict):
            detail = f"Dict len: {len(value)}"
            
        print(f"{key:<25} | {val_type:<15} | {detail}")

    print("-" * 70)
    
    # 4. Vérification de cohérence rapide
    print("\n--- Vérifications Rapides ---")
    n_nodes = loaded_data['num_nodes']
    print(f"Total Nœuds déclarés : {n_nodes:,}")
    
    # Vérifier si les indices max des tenseurs ne dépassent pas le nombre de nœuds
    max_idx_train = loaded_data['edge_index_train'].max().item()
    print(f"Indice Max dans Train  : {max_idx_train:,} (Doit être < {n_nodes})")
    
    if max_idx_train < n_nodes:
        print(" Cohérence des indices : OK")
    else:
        print(" ERREUR : Des indices dépassent la taille déclarée !")

except FileNotFoundError:
    print(f" Erreur : Le fichier '{FILE_PATH}' est introuvable.")
except Exception as e:
    print(f" Erreur lors de l'ouverture : {e}")

--- Chargement du fichier : /kaggle/working/amazon_kgcfrec_total_data.pt ---

 Fichier chargé avec succès !

CLÉ (Nom du variable)     | TYPE            | DÉTAIL / FORME
----------------------------------------------------------------------
num_nodes                 | int             | 691,559
num_users                 | int             | 392,089
num_items                 | int             | 215,039
num_relations             | int             | 4
edge_index_train          | Tensor          | Shape: [2, 19082728]
edge_index_val            | Tensor          | Shape: [2, 1001580]
edge_index_test           | Tensor          | Shape: [2, 1001580]
edge_index_kg             | Tensor          | Shape: [2, 5463300]
edge_type_kg              | Tensor          | Shape: [5463300]
user_encoder              | LabelEncoder    | Classes: 392,089
item_encoder              | LabelEncoder    | Classes: 215,039
entity_encoder            | LabelEncoder    | Classes: 84,431
relation_dict             | dict 